In [2]:
# Importações
from pybit.unified_trading import HTTP
import pandas as pd
import numpy as np
from datetime import datetime
import pandas_ta as ta
from enum import Enum

In [3]:
# Variáveis globais
cliente = HTTP()

data_inicio = '2023-07-01'
# data_fim = '2024-12-31'
data_fim = datetime.now()
saldo = 1000
cripto = 'SOLUSDT'
time_frame = '60'

In [4]:
# Busca de velas e somar todas as velas entre a data de início e a data de fim
data_inicio_timestamp = int( pd.to_datetime(data_inicio).timestamp() * 1000 )
data_fim_timestamp = int( pd.to_datetime(data_fim).timestamp() * 1000 )

velas_sem_estrutura = []

# loop para somar todas as velas entre a data de início e a data de fim
while data_inicio_timestamp < data_fim_timestamp:
    resposta = cliente.get_kline(symbol=cripto, interval=time_frame, limit=1000, start=data_inicio_timestamp)
    velas_sem_estrutura = velas_sem_estrutura + resposta['result']['list'][ : :-1]
    data_inicio_timestamp = int( velas_sem_estrutura[-1][0] ) + 1000

In [8]:
# Criação do DataFrame
colunas = ['tempo_abertura', 'abertura', 'maxima', 'minima', 'fechamento', 'volume', 'turnover']
df = pd.DataFrame(velas_sem_estrutura, columns=colunas)

In [10]:
# TIMESTAMP -> DATETIME
df['tempo_abertura'] = pd.to_datetime(df['tempo_abertura'].astype(np.int64), unit='ms')

# Tratamento dos dados para converter de texto para número com vírgula
df['abertura'] = df['abertura'].astype(float)
df['maxima'] = df['maxima'].astype(float)
df['minima'] = df['minima'].astype(float)
df['fechamento'] = df['fechamento'].astype(float)
df['volume'] = df['volume'].astype(float)
df['turnover'] = df['turnover'].astype(float)

df['ma_50'] = df['fechamento'].rolling(window=50).mean()
df['ma_200'] = df['fechamento'].rolling(window=200).mean()

In [8]:
def calcula_percentual_lucro_na_compra(preco_compra, preco_venda):
    percentual_lucro = ((preco_venda - preco_compra) / preco_compra) * 100
    # print('Percentual de lucro:', percentual_lucro, '%')
    return percentual_lucro

def calcula_percentual_perda_na_compra(preco_compra, preco_venda):
    percentual_perda = ((preco_compra - preco_venda) / preco_compra) * 100
    # print('Percentual de perda:', percentual_perda, '%')
    return percentual_perda

In [9]:
class EstadoDeTrade(Enum):
    DE_FORA = 0
    COMPRADO = 1
    VENDIDO = 2

estado_de_trade = EstadoDeTrade.DE_FORA

In [11]:
for i in range(199, len(df)):
    if estado_de_trade == EstadoDeTrade.DE_FORA:       
        # Avaliar condições de entrada seja na compra ou na venda
        media_rapida = df['ma_50'].iloc[i]
        media_lenta = df['ma_200'].iloc[i]

        if media_rapida > media_lenta: # Caso dê entrada:
            print('Condições de compra atendidas!')
            estado_de_trade = EstadoDeTrade.COMPRADO
            print('Comprou na vela de posição:', i)
            print('Comprou no tempo de abertura:', df['tempo_abertura'].iloc[i])
            preco_compra = df['abertura'].iloc[i]
            print('Preço de compra:', preco_compra)
            print('----------')

    if estado_de_trade == EstadoDeTrade.COMPRADO:
        # Avaliar condições de saída, seja no stop loss ou no alvo
        media_rapida = df['ma_50'].iloc[i]
        media_lenta = df['ma_200'].iloc[i]

        if media_rapida < media_lenta:
            print('Condições de saída da compra atendidas!')
            preco_venda = df['abertura'].iloc[i] 

            if preco_venda < preco_compra: # condição para avaliar perdas
                estado_de_trade = EstadoDeTrade.DE_FORA
                percentual_perda = calcula_percentual_perda_na_compra(preco_compra, preco_venda)
                saldo = saldo - (saldo * percentual_perda / 100)
                print('Stop loss na vela de posição:', i)
                print('Stop loss no tempo de abertura:', df['tempo_abertura'].iloc[i])
                print('Saldo atual:', saldo)
                print('----------')

            if preco_venda > preco_compra:
                # Caso de saída em alvo:
                estado_de_trade = EstadoDeTrade.DE_FORA
                percentual_lucro = calcula_percentual_lucro_na_compra(preco_compra, preco_venda)
                saldo = saldo + (saldo * percentual_lucro / 100)
                print('Saiu no alvo na vela de posição:', i)
                print('Saiu no alvo no tempo de abertura:', df['tempo_abertura'].iloc[i])
                print('Saldo atual:', saldo)
                print('----------')

print('Terminou o backtest!')
print('Saldo final:', saldo)

Condições de compra atendidas!
Comprou na vela de posição: 199
Comprou no tempo de abertura: 2023-07-09 07:00:00
Preço de compra: 21.758
----------
Condições de saída da compra atendidas!
Saiu no alvo na vela de posição: 474
Saiu no alvo no tempo de abertura: 2023-07-20 18:00:00
Saldo atual: 1175.3837668903393
----------
Condições de compra atendidas!
Comprou na vela de posição: 654
Comprou no tempo de abertura: 2023-07-28 06:00:00
Preço de compra: 24.669
----------
Condições de saída da compra atendidas!
Stop loss na vela de posição: 748
Stop loss no tempo de abertura: 2023-08-01 04:00:00
Saldo atual: 1114.444294765294
----------
Condições de compra atendidas!
Comprou na vela de posição: 929
Comprou no tempo de abertura: 2023-08-08 17:00:00
Preço de compra: 24.028
----------
Condições de saída da compra atendidas!
Stop loss na vela de posição: 1116
Stop loss no tempo de abertura: 2023-08-16 12:00:00
Saldo atual: 1089.9550910181626
----------
Condições de compra atendidas!
Comprou na v